- Thanks (upvote it): https://www.kaggle.com/rafiko1/1st-place-jigsaw-post-processing-example
- Thanks (upvote it) : https://www.kaggle.com/c/jigsaw-multilingual-toxic-comment-classification/discussion/160862

# Imports

In [ ]:
# General imports
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
WEIGHT = 1 # best to keep between 1 and 2 from the orignal authors

In [ ]:
submission = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sub_best = pd.read_csv('../input/minmax-highest-public-lb-9619/submission.csv')

In [ ]:
files_sub = [
    '../input/minmax-highest-public-lb-9619/submission.csv',
    '../input/minmax-ensemble-0-9526-lb/submission.csv',
    '../input/new-basline-np-log2-ensemble-top-10/submission.csv',
    '../input/stacking-ensemble-on-my-submissions/submission_mean.csv',
    '../input/analysis-of-melanoma-metadata-and-effnet-ensemble/ensembled.csv',
    '../input/eda-modelling-of-the-external-data-inc-ensemble/external_meta_ensembled.csv',
    '../input/submission-exploration/submission.csv',
    '../input/rc-fork-siim-isic-melanoma-384x384/sub_EfficientNetB2_384.csv',
    '../input/train-cv/submission.csv',
    '../input/triple-stratified-kfold-with-tfrecords/submission.csv',
    '../input/rank-then-blend/blend_sub.csv',
    '../input/analysis-of-melanoma-metadata-and-effnet-ensemble/blended_effnets.csv',
    '../input/cs0099/blended_effnets.csv',
    '../input/siim-isic-melanoma-classification-ensemble/submission.csv',
    '../input/minmax-ensemble-0-9526-lb/submission.csv',
    '../input/stacking-ensemble-on-my-submissions/submission_median.csv',
    '../input/cs0099/submission.csv',
    '../input/melanoma-pytorch-starter-efficientnet/submission.csv',
    '../input/power-of-metadata-xgboost-cnn-ensemble/submission.csv',
    '../input/melanoma-detection-with-pytorch/submission.csv',
    '../input/siim-isic-multiple-model-training-stacking/submission_mean.csv',
    '../input/post-processing-technique-c-f-1st-place-jigsaw/submission.csv',
    '../input/analysis-of-melanoma-metadata-and-effnet-ensemble/meta_simplified_img_data.csv',
    '../input/eda-modelling-of-the-external-data-inc-ensemble/external_tabular_predicts.csv',
    '../input/public-blender-with-rank-data-0-946/submission.csv',
    '../input/cs0099/submission_jig.csv',
    '../input/blending-submissions-0-966/submission.csv',
    '../input/incredible-tpus-finetune-effnetb0-b6-at-once/submission_models_blended.csv',
    '../input/dark-magic-blending-0-9603/submission.csv',
    '../input/cs0099/submission_mean.csv',
    '../input/cs0099/submission.csv',
    '../input/inference-melanoma-crazy-fast/submission.csv',
    '../input/siim-isic-efficientnet-b6-single-model-lb-0-9475/submission.csv',
    '../input/simple-oof-ensembling-methods-for-classification/submission_bo.csv',
    '../input/inference-melanoma-crazy-fast/submission.csv',
    '../input/let-the-magic-begin/stacking.csv'
    
]
files_sub = sorted(files_sub)
print(len(files_sub))
files_sub

In [ ]:
for file in files_sub:
    test[file.replace(".csv", "")] = pd.read_csv(file).sort_values('image_name')["target"]
test['id'] = test.index

In [ ]:
# Derive the given sub increases or decreases in score
test["diff_good1"] =  test['../input/new-basline-np-log2-ensemble-top-10/submission'] - test['../input/stacking-ensemble-on-my-submissions/submission_median']


test["diff_good2"] =  test['../input/eda-modelling-of-the-external-data-inc-ensemble/external_meta_ensembled'] - test['../input/let-the-magic-begin/stacking']
    
    
test["diff_good3"] = test['../input/minmax-ensemble-0-9526-lb/submission'] - test['../input/rc-fork-siim-isic-melanoma-384x384/sub_EfficientNetB2_384']
    
    
test["diff_good4"] = test['../input/analysis-of-melanoma-metadata-and-effnet-ensemble/ensembled'] - test['../input/rank-then-blend/blend_sub']
    

# test["diff_good5"] = test['../input/submission-exploration/submission'] - test['../input/new-basline-np-log2-ensemble-top-10/submission']


test["diff_good6"] = test['../input/simple-oof-ensembling-methods-for-classification/submission_bo'] - test['../input/siim-isic-efficientnet-b6-single-model-lb-0-9475/submission']


test["diff_bad1"] = test['../input/triple-stratified-kfold-with-tfrecords/submission'] - test['../input/train-cv/submission']

In [ ]:
test["sub_best"] = test['../input/minmax-highest-public-lb-9619/submission']

col_comment = ["id", "image_name", "patient_id", "sub_best"]
col_diff = [column for column in test.columns if "diff" in column]
test_diff = test[col_comment + col_diff].reset_index(drop=True)

test_diff["diff_avg"] = test_diff[col_diff].mean(axis=1) # the mean trend

In [ ]:
# Apply the post-processing technique in one line (as explained in the pseudo-code of my post.
test_diff["sub_new"] = test_diff.apply(lambda x: (1+WEIGHT*x["diff_avg"])*x["sub_best"] if x["diff_avg"]<0 else (1-WEIGHT*x["diff_avg"])*x["sub_best"] + WEIGHT*x["diff_avg"] , axis=1)

In [ ]:
submission["target"] = sub_best["target"]
submission.head()

In [ ]:
test_diff.head()

In [ ]:
submission.loc[test["id"], "target"] = test_diff["sub_new"].values

In [ ]:
submission.to_csv("submission.csv", index=False)
submission.head()

In [ ]:
plt.hist(submission.target,bins=100)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
sns.distplot(submission['target'], hist_kws={
                 'rwidth': 0.75,
                 'edgecolor': 'black',
                 'alpha': 0.3
             }, color='#C3073F')
ax.set_title('Final Predictions')
plt.show()

# LB 96.48